# Workflow model evaluatie: Deel 2 Validatie

Laatste aanpassing: 25 nov 2024 <br>
Contact: Petra Hulsman <br>

## Context

Binnen HDSR worden hydraulische modellen gebruikt voor het beantwoorden van verschillende vraagstukken. Bijvoorbeeld: Zitten er knelpunten in het systeem bij een T=1 zomer afvoersituatie? Hoeveel ruimte is er beschikbaar voor NVOs? Tot welke diepte moet minimaal gebaggerd worden? Welke dimensies moeten nieuwe kunstwerken bij voorkeur hebben om knelpunten te kunen voorkomen? Welke gebieden zijn kwetsbaar voor inundatie? Momenteel (anno begin 2024) wordt voornamelijk Sobek gebruikt om debieten en waterniveaus in primaire watergangen te modelleren. Het is van groot belang dat onze modellen betrouwbaar zijn.

In deze notebook is een workflow voor de model evaluatie opgezet. Deze bestaat uit drie delen:
1. Verificatie: model resultaten vs. beheerregister (BR)
2. Validatie: model resultaten vs. puntmetingen
3. Model vergelijking: model A vs. model B/satelliet data (n.b.: satelliet data zijn vaak ook het resultaat van modellen)

Deze workflow kan gebruikt worden om de betrouwbaarheid van onze modellen te kunnen bepalen en om model onnauwkeurigheden te kunnen identificeren en verbeteren. Deze evaluatie is sterk afhankelijk van de betrouwbaarheid van geobserveerde data, namelijk veld observaties en satelliet data die hiervoor gebruikt worden. Het is daarom erg belangrijk om te onthouden dat er niet alleen onzekerheden zijn in het model maar ook in de geobserveerde data. Daarom wordt aanbevolen kritisch naar de resultaten te kijken voordat een oordeel getroffen wordt.

## Uitvoer

Deze code werkt nu met de virtuale omgeving "Petra_arcpy" op de rekenserver reken05. Momenteel is een handmatige na-controle aan te raden om te verifieren of alle ruimtelijke koppelingen tussen model en data goed is gegaan.

Belangrijke informatie:
De FEWS-WIS data is m.b.v. een aparte script iets bewerkt voor dat het hier gebruikt wordt (vandaar ook de subscript "edited" in de bestandsnamen). Hierbij is op locaties waar er meerere pompen zijn, het totaal debiet toegevoegd.

### Waterniveau: RUPROF
<br>
<u>Variabele</u>:  <br>
- Waterniveau voor T=1 in de zomer  <br>
<u>Aanpak</u>:  <br>
- Verschil tussen 1) data (FEWS-WIS), en 2) model (RUPROF) <br>
<u>Randvoorwaarden</u>: <br>
- Minimaal 3 jaar aan geobserveerde data beschikbaar (niet per se aaneengesloten) voor het berekenen van het T=1 debiet. <br>
- Meetpunt dichtbij een peilschijdend kunstwerk (stuw, pomp of duiker)? Dan moeten het modelpunt en meetpunt zich in hetzelfde insteekvlak bevinden. <br>
<u>Output</u>: <br>
- Shapefile (polypoints): verschil tussen data en model <br>
- Figuren: verschil per observatie punt en maximum verschil per afvoergebied

In [ ]:
import os
import numpy as np
import functies_punt_WL_T1
class paths():
    # Input data
    root           = os.getcwd()    
    df_WL_WISdata  = os.path.join(root, 'Input','FEWS-WIS','WL_2000_2023_werkfilter.csv') # FEWS-WIS data
    df_WL_WISsites = os.path.join(root, 'Input','FEWS-WIS','ow_ws.csv') # FEWS-WIS metadata voor meetpunten
    df_model       = os.path.join(root, 'Input','Sobek output','SOBEK_WL_150724.csv') # gemodelleerd debiet op basis van Sobek (op t=0 uit RUPROF)
    shp_node       = os.path.join(root,"GIS input",'3b_nod_joined.shp') # shapefile van de model knopen, export uit Sobek en ge-joined met BR-peilgebieden in ArcMAP
    shp_afvoer     = os.path.join(root,"GIS input","BR_VS_Afvoergebieden.shp") # shapefile van de afvoergebieden
    shp_insteek    = os.path.join(root,"GIS input","BR-VS_Insteekvlak.shp") # shapefile van het insteekvlak
    shp_reach      = os.path.join(root,"GIS input","RchSegments_Qstat.shp") # shapefile van de reach segments (watergangen), export uit Sobek
    shp_regios     = os.path.join(root,"GIS input","HDSR_regios.shp") # shapefile van HDSR regios
    
    # Output data
    gdf_data      = os.path.join(root,"GIS output","WL_T1_DATA.shp") # shapefile met de gemeten data (obv FEWS-WIS)
    gdf_model     = os.path.join(root,"GIS output","WL_T1_RUPROF.shp") # shapefile met de Sobek data
    gdf_joined    = os.path.join(root,"GIS output","WL_T1_RUPROF_DATA.shp") # shapefile met gemeten & gemodelleerd waterniveau
    fig           = os.path.join(root,"Output",'2_fig_WL_T1.png') # figuur met de verschillen in waterniveau
    fightml       = os.path.join(root,"Output",'2_fig_WL_T1.html') # figuur met de verschillen in waterniveau
    
months = [5,6,7,8,9] # maanden in het jaar om statistiek over te berekenen
years  = np.arange(2000,2023)

functies_punt_WL_T1.main(paths,months,years)

### Debiet: RUPROF
<br>
<u>Variabele</u>:  <br>
- Debiet voor T=1 in de zomer  <br>
<u>Aanpak</u>:  <br>
- Verschil tussen 1) data (FEWS-WIS), en 2) model (RUPROF) <br>
<u>Randvoorwaarden</u>: <br>
- Debiet data bij gemalen en debietmeters (overige kunstwerken zijn momenteel uitgesloten). <br>
- Debieten bij gemalen zijn vooraf opgeteld (in een aparte script). <br>
- Minimaal 3 jaar aan geobserveerde data beschikbaar (niet per se aaneengesloten) voor het berekenen van het T=1 debiet. <br>
<u>Output</u>: <br>
- Shapefile (polypoints): verschil tussen data en model <br>
- Figuren: verschil per observatie punt en maximum verschil per afvoergebied



In [ ]:
import os
import numpy as np
import functies_punt_Q_T1

class paths():
    # Input data
    root          = os.getcwd()    
    df_Q_model    = os.path.join(root, 'Input','Sobek output','SOBEK_Q_150724.csv') # gemodelleerd debiet op basis van Sobek (op t=0 uit RUPROF)
    # verwijder in df_Q_model de exponent in de eenheid (door m3/s te typen in het csv bestand); de exponent zorgt namelijk voor een foutmelding bij het inlezen van het bestand
    df_Q_WISdata  = os.path.join(root, 'Input','FEWS-WIS','Q_kunstwerken_2000_2023_werkfilter_edited.csv') # FEWS-WIS data bij kunstwerken
    df_Q_WISsites = os.path.join(root, 'Input','FEWS-WIS','ow_sl_edited.csv') # FEWS-WIS metadata voor meetpunten
    shp_peil      = os.path.join(root,"GIS input","BR-VS_Peilgebieden.shp") # shapefile van de peilgebieden
    shp_reach     = os.path.join(root,"GIS input","RchSegments.shp") # shapefile van de reach segments (watergangen), export uit Sobek
    shp_afvoer    = os.path.join(root,"GIS input","BR_VS_Afvoergebieden.shp") # shapefile van de afvoergebieden
    shp_insteek   = os.path.join(root,"GIS input","BR-VS_Insteekvlak.shp") # shapefile van het insteekvlak
    shp_regios    = os.path.join(root,"GIS input","HDSR_regios.shp") # shapefile van HDSR regios
    
    # Output data
    gdf_data      = os.path.join(root,"GIS output","Q_T1_DATA.shp") # shapefile met de gemeten debiet data (obv FEWS-WIS)
    gdf_model     = os.path.join(root,"GIS output","Q_T1_RUPROF.shp") # shapefile met de Sobek data
    gdf_joined    = os.path.join(root,"GIS output","Q_T1_RUPROF_DATA.shp") # shapefile met gemeten & gemodelleerd debiet
    fightml       = os.path.join(root,"Output",'2_fig_Q_T1.html') # figuur met de verschillen in debiet


months = [5,6,7,8,9] # maanden in het jaar om statistiek over te berekenen
years  = np.arange(2000,2023)
functies_punt_Q_T1.main(paths, months, years)
    

### Debiet: Sobek continue model (2010 - 2021)
<br>
<u>Variabele</u>:  <br>
- Gemiddeld debiet in primaire watergang  <br>
<u>Aanpak</u>:  <br>
- Verschil tussen 1) data (FEWS-WIS), en 2) model (Sobek model) <br>
<u>Randvoorwaarden</u>: <br>
- Validatie over de periode 2010 - 2021 <br>
- Minimaal 1 jaar aan geobserveerde data beschikbaar (niet per se aaneengesloten) <br>
- Statistieken: verschil gemiddelde waarde  <br>
<u>Output</u>: <br>
- Shapefile (polypoints): verschil tussen data en model gemiddeld over een periode  <br>
- Figuren: verschil per observatie punt en maximum verschil per afvoergebied

In [ ]:
import os
import numpy as np
import functies_punt_Qseries

class paths():
    # Input data
    root          = os.getcwd()    
    data_sobek    = os.path.join(root,"Input","Sobek output","MOD_ADJ_V4.lit") # folder Sobek HIS bestanden
    shp_reach     = os.path.join(root,"GIS input","RchSegments_Qstat.shp") # shapefile van de reach segments (watergangen), export uit Sobek
    shp_afvoer    = os.path.join(root,"GIS input","BR_VS_Afvoergebieden.shp") # shapefile van de afvoergebieden
    df_Q_WISdata  = os.path.join(root, 'Input','FEWS-WIS','Q_hourly_pomp_ADCP_2010_2023_werkfilter_edited.csv') # FEWS-WIS data bij kunstwerken
    df_Q_WISsites = os.path.join(root, 'Input','FEWS-WIS','ow_sl_edited.csv') # FEWS-WIS metadata voor meetpunten
    shp_insteek   = os.path.join(root,"GIS input","BR-VS_Insteekvlak.shp") # shapefile van het insteekvlak
    
    # Output data
    gdf_data      = os.path.join(root,"GIS output","Q_gem_DATA.shp") # shapefile met gemeten debiet
    gdf_model     = os.path.join(root,"GIS output","Q_gem_Sobek.shp") # shapefile met gemodelleerd debiet
    gdf_joined    = os.path.join(root,"GIS output","Q_gem_Sobek_DATA.shp") # shapefile met gemeten & gemodelleerd debiet
    fightml       = os.path.join(root,"Output",'2_fig_Q_gem.html') # figuur met de verschillen in debiet

months        = np.arange(1,13) # maanden in het jaar om statistiek over te berekenen
years         = np.arange(2010,2022) # jaren om statistiek over te berekenen
functies_punt_Qseries.main(paths, months, years)

### Stroomrichting: Sobek continue model (2010 - 2021)
<br>
<u>Variabele</u>:  <br>
- Stroomrichting in primaire watergang  <br>
<u>Aanpak</u>:  <br>
- Verschil tussen 1) data (FEWS-WIS), en 2) model (Sobek model) <br>
<u>Randvoorwaarden</u>: <br>
- Validatie over de periode 2010 - 2021 <br>
- Statistieken: (alleen visueel)  <br>
<u>Output</u>: <br>
- Figuren: tijdreeks op locaties waar de stroomrichting veranderd

In [ ]:
import os
import numpy as np
import functies_punt_Qdir

class paths():
    # Input data
    root          = os.getcwd()    
    data_sobek    = os.path.join(root,"Input","Sobek output","MOD_ADJ_V4.lit") # folder Sobek HIS bestanden
    shp_reach     = os.path.join(root,"GIS input","RchSegments_Qstat.shp") # shapefile van de reach segments (watergangen), export uit Sobek
    shp_afvoer    = os.path.join(root,"GIS input","BR_VS_Afvoergebieden.shp") # shapefile van de afvoergebieden
    df_Q_WISdata  = os.path.join(root, 'Input','FEWS-WIS','Q_hourly_pomp_ADCP_2010_2023_werkfilter_edited.csv') # FEWS-WIS data bij kunstwerken
    df_Q_WISsites = os.path.join(root, 'Input','FEWS-WIS','ow_sl_edited.csv') # FEWS-WIS metadata voor meetpunten
    shp_insteek   = os.path.join(root,"GIS input","BR-VS_Insteekvlak.shp") # shapefile van het insteekvlak
    
    # Output data
    fightml       = os.path.join(root,"Output",'2_fig_Q_dir.html') # figuur met de debieten op locaties waar de stroomrichting veranderd

months        = np.arange(1,13) # maanden in het jaar om statistiek over te berekenen
years         = np.arange(2010,2022) # jaren om statistiek over te berekenen
functies_punt_Qdir.main(paths, months, years)